# Count Based Word Representation

- 자연어 처리에서 텍스트를 표현하는 방법으로는 여러가지 방법이 있음

- 정보 검색과 텍스트 마이닝 분야에서 주로 사용되는

    - 카운트 기반의 텍스트 표현 방법인 DTM(Document Term Matrix)

    - TF-IDF(Term Frequency-Inverse Document Frequency)

- 텍스트를 위와 같은 방식으로 수치화를 하고 나면,

- 어떤 단어가 특정 문서 내에서 얼마나 중요한 것인지를 나타내거나, 문서의 핵심어 추출, 검색 순위 결정, 문장들 간의 유사도를 구하는 등의 용도로 사용할 수 있음

### 1. 다양한 단어의 표현 방법

##### 1-1. 단어의 표현 방법

- 단어의 표현 방법은 국소 표현(Local Representation) 방법과 분산 표현(Distributed Representation)방법으로 나뉨

- 국소 표현 방법은 해당 단어 자체만 보고 특정 값을 맵핑하여 단어를 표현하는 방법

- 분산 표현 방법은 그 단어를 표현하고자 주변을 참고하여 단어를 표현하는 방법

- 예를 들어 puppy, cute, lovely라는 단어에 1번, 2번, 3번의 숫자를 맵핑하여 부여하면 국소 표현

- puppy라는 단어 근처에 주로 cute, lovely라는 단어가 자주 등장하므로 puppy는 cute, lovely한 느낌이다로 단어를 정의하면 분산 표현

- 비슷한 의미로 국소 표현을 이산 표현이라고, 분산 표현은 연속 표현이라고도 함

##### 1-2. 단어 표현의 카테고리화

- 이번 챕터의 Bag of Words는 국소 표현으로 단어의 빈도수를 카운트하여 수치화

- BoW와 그의 확장인 DTM(또는 TDM)에 대해 학습하고

- 이러한 빈도수 기반 단어 표현에 단어의 중요도에 따른 가중치를 줄 수 있는 TF-IDF에 대해서 학습

### 2. Bag of Words(BoW)

- 단어의 등장 순서를 고려하지 않는 빈도수 기반의 단어 표현 방법

##### 2-1. Bag of Words란?

- 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터 수치화 표현 방법

- BoW 만드는 과정

    - 각 단어에 고유한 정수 인덱스를 부여(단어 집합 생성)

    - 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듦

In [3]:
from konlpy.tag import Okt

okt = Okt()

def build_bag_of_words(document):
    document = document.replace('.', '')
    tokenized_document = okt.morphs(document)
    
    word_to_index = {}
    bow = []
    
    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            # BoW에 전부 기본값 1을 넣음
            bow.insert(len(word_to_index) - 1, 1)
        else:
            # 재등장하는 단어의 인덱스
            index = word_to_index.get(word)
            # 재등장한 단어는 해당하는 인덱스의 위치에 1을 더함
            bow[index] = bow[index] + 1
    
    return word_to_index, bow

In [4]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."

vocab, bow = build_bag_of_words(doc1)

print('vocabulary: ', vocab)
print('bag of words vector: ', bow)

vocabulary:  {'정부': 0, '가': 1, '발표': 2, '하는': 3, '물가상승률': 4, '과': 5, '소비자': 6, '느끼는': 7, '은': 8, '다르다': 9}
bag of words vector:  [1, 2, 1, 1, 2, 1, 1, 1, 1, 1]


- BoW는 각 단어가 등장한 횟수를 수치화하는 텍스트 표현 방법이므로

- 주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서의 성격을 판단하는 작업에 쓰임

- 즉, 분류 문제나 여러 문서 간의 유사도를 구하는 문제에 주로 쓰임

- '달리기', '체력', '근력'과 같은 단어가 자주 등장하면 해당 문서를 체육 관련 문서로 분류

- '미분', '방정식', '부등식'과 같은 단어가 자중 등장한다면 수학 관련 문서로 분류

##### 2-3. CountVectorizer 클래스로 BoW 만들기

- scikit learn에서는 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스를 지원

In [7]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["you know I want your love. becauses I love you."]
vector = CountVectorizer()

# 코퍼스로부터 각 단어의 빈도수를 기록
print('bag of words vector: ', vector.fit_transform(corpus).toarray())

# 각 단어의 인덱스가 어떻게 부여되었는지를 출력
print('vocabulary: ', vector.vocabulary_)

bag of words vector:  [[1 1 2 1 2 1]]
vocabulary:  {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'becauses': 0}


- CountVectorizer가 기본적으로 길이가 2이상인 문자에 대해서만 토큰으로 인식하기 때문에 'I' 사라짐

- 주의할 점은 CountVectorizer는 단지 띄어쓰기만을 기준으로 단어를 자르는 낮은 수준의 토큰화로 BoW를 만듦

- 위의 예시 문장인 "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다"에서

- 물가상승률이 각자 다른 인덱스에서 1이라는 빈도의 값을 갖게 되는 문제

##### 2-4. 불용어를 제거한 BoW 만들기

- BoW를 사용한다는 것은 그 문서에서 각 단어가 얼마나 자주 등장했는지를 보겠다는 것이며

- 각 단어에 대한 빈도수를 수치화 하겠다는 것은 결국 텍스트 내에서 어떤 단어들이 중요한지를 보고싶다는 의미

- BoW를 만들 때 불용어를 제거하는 일은 자연어 처리의 정확도를 높이기 위해서 선택할 수 있는 전처리 기법

- 사용자가 직접 정의한 불용어 사용

In [8]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])

print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)

bag of words vector : [[1 1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


- CountVectorizer에서 제공하는 자체 불용어 사용

In [9]:
text = ["Family is not an important thing. It's everything."]
vect = CountVectorizer(stop_words="english")

print('bag of words vector :',vect.fit_transform(text).toarray())
print('vocabulary :',vect.vocabulary_)


bag of words vector : [[1 1 1]]
vocabulary : {'family': 0, 'important': 1, 'thing': 2}


- NLTK에서 지원하는 불용어 사용

In [13]:
from nltk.corpus import stopwords

In [14]:
text = ["Family is not an important thing. It's everything."]
stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)

print('bag of words vector :',vect.fit_transform(text).toarray()) 
print('vocabulary :',vect.vocabulary_)


bag of words vector : [[1 1 1 1]]
vocabulary : {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}


### 3. 문서 단어 행렬(Document-Term Matrix, DTM)

- 서로 다른 문서들의 BoW들을 결합한 표현 방법은 문서 단어 행렬 DTM

- 행과 열은 반대로 선택하면 TDM

##### 3-1. DTM의 표기법

- DTM이란 다수의 문서에서 등장하는 각 단어들의 빈도를 행렬로 표현한 것을 말함

- 쉽게 생각하면 각 문서에 대한 BoW를 하나의 행렬로 만든 것

##### 3-2. DTM의 한계

- 희소 표현

    - 원-핫 벡터는 단어 집합의 크기가 벡터의 차원이 되고 대부분의 값이 0이 되는 벡터로

    - 공간적 낭비와 계산 리소스를 증가시킬 수 있다는 단점을 가짐

    - DTM도 마찬가지로 각 문서 벡터의 차원은 전체 단어 집합의 크기이므로 희소 행렬이 됨

- 단순 빈도 수 기반 접근

    - 여러 문서에 등장하는 모든 단어에 대해 빈도 표기를 할 때 때로는 한계를 가지기도 함

    - 불용어인 the는 어떤 문서이든 자주 등장하므로 여러 문서에서 the의 빈도수가 높게 나왔다고 해서 유사한 문서라고 판단해서는 안 됨

    - 그렇다면 DTM에 불용어와 중요한 단어에 대해서 가중치를 줄 수 있는 TF-IDF 방식의 중요성

### 4. TF-IDF(Term Frequenct-Inverse Document Frequency)

##### 4-1. TF-IDF

- TF-IDF는 단어의 빈도와 역 문서 빈도(문서의 빈도에 특정 식을 취함)를 사용하여 DTM 내의 각 단어들마다 중요한 정도를 가중치로 주는 방법

- TF-IDF는 주로 문서의 유사도를 구하는 작업, 검색 시스템에서 검색 결과의 중요도를 정하는 작업, 문서 내에서 특정 단어의 중요도를 구하는 작업 등에 쓰일 수 있음

- TF-IDF는 TF와 IDF를 곱한 값. 문서를 d, 단어를 t, 문서의 총 개수를 n 이라고 하면

    - tf(d, t): 특정 문서 d에서의 특정 단어 t의 등장 횟수

        - DTM에서 각 단어들이 가진 값

    - df(t): 특정 단어 t가 등장한 문서의 수

        - 특정 단어가 각 문서에서 몇 번 등장한지가 아니라 등장한 문서의 수에만 관심

    - idf(t): df(t)에 반비례하는 수
<math xmlns="http://www.w3.org/1998/Math/MathML" display="block">
  <mi>i</mi>
  <mi>d</mi>
  <mi>f</mi>
  <mo stretchy="false">(</mo>
  <mi>t</mi>
  <mo stretchy="false">)</mo>
  <mo>=</mo>
  <mi>l</mi>
  <mi>o</mi>
  <mi>g</mi>
  <mo stretchy="false">(</mo>
  <mfrac>
    <mi>n</mi>
    <mrow>
      <mn>1</mn>
      <mo>+</mo>
      <mi>d</mi>
      <mi>f</mi>
      <mo stretchy="false">(</mo>
      <mi>t</mi>
      <mo stretchy="false">)</mo>
    </mrow>
  </mfrac>
  <mo stretchy="false">)</mo>
</math>

        - DF의 역수를 취하면서 분모에 1을 더해주고 log를 취함

        - log를 사용하지 않으면 n이 커질 수록 IDF의 값이 기하급수적으로 커지게 되므로

        - 희귀 단어들은 빈도가 적기 때문에 log가 없으면 엄청난 가중치가 부여되는 현상이 나타남

        - 1을 더해주는 이유는 단어가 문서에서 등장하지 않아 분모가 0이 되는 상황을 방지하기 위해

- TF-IDF는 모든 문서에서 자주 등장하는 단어는 중요도가 낮다고 판단하며

- 특성 문서에서만 자주 등장하는 단어는 중요도가 높다고 판단

- TF-IDF 값이 낮으면 중요도가 낮으며, 높으면 중요도도 큼
      

##### 4-2. 파이썬으로 TF-IDF 직접 구현

In [19]:
import pandas as pd
from math import log

docs = [
    "먹고 싶은 사과", 
    "먹고 싶은 바나나", 
    "길고 노란 바나나 바나나", 
    "저는 과일이 좋아요"
]

vocab = list(set(w for doc in docs for w in doc.split()))
vocab.sort()

In [20]:
# 총 문서의 수
N = len(docs)

def tf(t, d):
    
    return d.count(t)

def idf(t):
    df = 0
    for doc in docs:
        df += t in doc
    
    return log(N/(df+1))

def tfidf(t, d):
    
    return tf(t, d) * idf(t)

In [22]:
# TF

result = []

for i in range(N):
    result.append([])
    d = docs[i]
    
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tf(t, d))
        
tf_ = pd.DataFrame(result, columns=vocab)

display(tf_)

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0,0,0,1,0,1,1,0,0
1,0,0,0,1,1,0,1,0,0
2,0,1,1,0,2,0,0,0,0
3,1,0,0,0,0,0,0,1,1


In [23]:
# IDF

result = []

for j in range(len(vocab)):
    t = vocab[j]
    result.append(idf(t))

idf_ = pd.DataFrame(result, index=vocab, columns=['IDF'])

display(idf_)

,IDF
과일이,0.693147
길고,0.693147
노란,0.693147
먹고,0.287682
바나나,0.287682
사과,0.693147
싶은,0.287682
저는,0.693147
좋아요,0.693147


In [24]:
result = []

for i in range(N):
    result.append([])
    d = docs[i]
    
    for j in range(len(vocab)):
        t = vocab[j]
        result[-1].append(tfidf(t, d))

tfidf_ = pd.DataFrame(result, columns=vocab)

display(tfidf_)

,과일이,길고,노란,먹고,바나나,사과,싶은,저는,좋아요
0,0.000000,0.000000,0.000000,0.287682,0.000000,0.693147,0.287682,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.287682,0.287682,0.000000,0.287682,0.000000,0.000000
2,0.000000,0.693147,0.693147,0.000000,0.575364,0.000000,0.000000,0.000000,0.000000
3,0.693147,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.693147,0.693147


- TF-IDF의 가장 기본적인 식

- 실제 TF-IDF 구현을 제공하는 식은 조금씩 상이해서 조정된 식을 사용함

- 조정된 식을 사용하는 이뉴는 기본적인 식을 바탕으로한 구현에 몇 가지 문제점이 있기 때문

- 만약 전체 문서의 수 n이 4인데 df(t)의 값이 3이면 분모가 1이 되면서 idf(d, t)의 값이 0

- 결국 가중치의 역할을 수행하지 못 함


##### 4-3. 사이킷런을 이용한 DTM과 TF-IDF 실습

In [1]:
from sklearn.feature_extraction.text import CountVectorizer

# DTM

corpus = [
    "you know I want your love", 
    "I like  you", 
    "what should I do"
]
vector = CountVectorizer()

print(vector.fit_transform(corpus).toarray())
print(vector.vocabulary_)

[[0 1 0 1 0 1 0 1 1]
 [0 0 1 0 0 0 0 1 0]
 [1 0 0 0 1 0 1 0 0]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF
# 사이킷런의 경우 IDF의 로그항의 분자에 1을 더해주고, 로그항에 1을 더해고, L2 정규화로 값을 조정함

corpus = [
    "you know I want your love", 
    "I like  you", 
    "what should I do"
]
tfidfv = TfidfVectorizer().fit(corpus)

print(tfidfv.transform(corpus).toarray())
print(tfidfv.vocabulary_)

[[0.         0.46735098 0.         0.46735098 0.         0.46735098
  0.         0.35543247 0.46735098]
 [0.         0.         0.79596054 0.         0.         0.
  0.         0.60534851 0.        ]
 [0.57735027 0.         0.         0.         0.57735027 0.
  0.57735027 0.         0.        ]]
{'you': 7, 'know': 1, 'want': 5, 'your': 8, 'love': 3, 'like': 2, 'what': 6, 'should': 4, 'do': 0}
